In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
# python version: 3.11.9

In [7]:
# presets, functions

# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
Z = 6
A = 12
mass_nucleon = 0.938273 # proton mass?
# mass_nucleus = 11.178
mass_nucleus = A*0.931494
alpha_fine = 1/137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

# map from data set index to data set name (author, year, etc.)
dataSet_to_name = {1:"Barreau:1983ht", 2:"O'Connell:198", 3:"Sealock:1989nx", 4:"Baran:1988tw", 5:"Bagdasaryan:1988hp", 6:"Dai - HallA:2019da", 
    7:"Arrington:1995hs", 8:"Day:1993md", 9:"Arrington:1998psnoCC", 10:"Gaskell:2008", 11:"Whitney:1974hr", 12:"AlsamiJan05", 13:"VaheJun07", 
    14:"Gomez74", 15:"Fomin", 16:"Yamaguchi73", 17:"Ryan84", 18:"Cyzyk:1963zz", 19:"Bounin63", 20:"Photo-Daphne", 
    # 21:"Spamer70", 
    21:"Antony-Spies:1970jjs",
    22:"Goldemberg64", 
    23:"DeForrest65", 24:"Donnelly68", 25:"Barreau:French",26:"GENIE:LFG",27:"GENIE:SUSA"}

# # map from data set index to normalization factor
# dataSet_to_normalization = {1:0.99185, 2:0.97869, 3:1.0315, 4:0.99241, 5:0.98777, 6:1.0108, 7:0.97427, 8:1.0071, 9:0.98884, 10:0.99340, 11:1.0149, 
#                 12:0.99812, 13:1.0029, 14:1.0125, 15:1.0046, 16:1.0019, 
#                 # 17:1.0517, 
#                 17:1.1, 
                
#                 # 18:1.0, 
#                 18:1.2,
#                 19:1.1500, 20:0.99754, 21:0.95, 22:1.1, 23:0.9, 24:1.0, 
#                 25:0.99185,26:1.0,27:1.0}
# # map from data set index to normalization error
# dataSet_to_normError = {1:0.24320E-02, 2:0.86008E-02, 
#                         # 3:0.48305E-02,
#                         3:0.1,
#                         4:0.45501E-02, 5:0.82797E-02, 6:0.52969E-02, 7:0.13229E-01, 8:0.33307E-02, 
#         9:0.34432E-02, 10:0.50742E-02, 11:0.15258E-01, 12:0.67079E-03, 13:0.69905E-03, 14:0.14875E-01, 15:0.30856E-02, 16:0.29024E-02, 
#         17:0.13049E-01, 
#         18:0.2, 
#         19:0.23, 20:0.10513, 21:0.25, 22:0.1, 23:0.1, 24:0.1, 25:0.002432, 26:0.0, 27:0.0}

# March 12:
dataSet_to_normalization = {
    1:0.9701,
    2:0.9507,
    3:1.075,
    4:1.002,
    5:0.9124,
    6:1.0149,
    7:0.9502,
    8:1.026,
    9:0.9888,
    10:0.9846,
    11:1.034,
    12:0.9956,
    13:1.011,
    14:1.0125,
    15:1.0046,
    16:1.0019,
    17:1.1255,
    18:1.2369,
    19:1.22,
    20:1.0,
    21:0.95,
    22:1.1095,
    23:1.0931,
    24:1.03,
    25:0.85,
    26:1.0
}

dataSet_to_normError = {
    1:0.002,
    2:0.0006,
    3:0.0037,
    4:0.0033,
    5:0.0083,
    6:0.0060,
    7:0.0096,
    8:0.0035,
    9:0.0034,
    10:0.0066,
    11:0.013,
    12:0.0009,
    13:0.0018,
    14:0.0149,
    15:0.0031,
    16:0.0029,
    17:0.0130,
    18:0.2,
    19:0.23,
    20:0,
    21:0.25,
    22:0.1,
    23:0.1,
    24:0.02,
    25:0.02,
    26:0.02,
}
#

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def linear_model(x, a, b):
    return a * x + b

def Vcoul(A=12,Z=6):
    HBARC  = 0.197327      # in GeV.fm
    ALPHA  = 1.0/137.0
    C_ASTE = 0.775
    R0     = 1.1*A**(1./3.) + 0.86*A**(-1./3.)

    # Coulomb potential at the center of the nucleus
    V0  = (3./2.)*ALPHA*HBARC*(Z-1.)/R0  # in GeV

    # Average potential
    V  = C_ASTE*V0      # from Eur. Phys. J. A26 (2005) 167
    return V

def prepare_df(filepath):
    # read the csv file:
    df = pd.read_csv(filepath)

    if 'error' not in df.columns:
        df['error']=0.0


    # calculate normalized cross section:
    if 'dataSet' not in df.columns:
        df['normalization']=1
        df['normError']=0
    else:
        df["normalization"]=df["dataSet"].map(dataSet_to_normalization)
        df["normError"]=df["dataSet"].map(dataSet_to_normError)
    df['normCross'] = df['cross'] * df['normalization']
    

    
    # system_err = 0.02
    system_err = 0.0

    df['error'] = np.sqrt(df['error']**2 + ((system_err*df['cross'])**2))
    df['normCrossError']=df['normCross']*np.sqrt((df['error']/df['cross'])**2+(df['normError']/df['normalization'])**2)
    
    # calculate the kinematic variables:
    df["ThetaRad"]=df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"]=(np.sin(df["ThetaRad"]/2))**2
    df["cos2(T/2)"]=(np.cos(df["ThetaRad"]/2))**2
    df["tan2(T/2)"]=(np.tan(df["ThetaRad"]/2))**2

    # df["nu_elastic"]=df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12)
    # df["Ex"]=df["nu"]-df["nu_elastic"]
    df["nuel"]=df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12)
    df["Ex"]=df["nu"]-df["nuel"]
    df["Ex"] = df["nu"] - (df["E0"]-df["E0"]/(1+2*df["E0"]*df["sin2(T/2)"]/mass_C12))

    # R = 2.894, Z = 6, A = 12
    df["R"]=1.1*(df["A"])**(1/3)+0.86/((df["A"])**(1/3))
    

    # Data:
    df["Veff"]=Vcoul(A=12,Z=6)

    # GENIE:
    # df["Veff"]=0.0
    
    
    df["Eeff"]=df["E0"]+df["Veff"]
    df["Ep"]=df["E0"]-df["nu"]
    df["Ep_eff"]=df["Ep"]+df["Veff"]

    df["F2foc"]=(df["Eeff"]/df["E0"])**2
    df["Q2"]=4*df["E0"]*(df["Ep"])*df["sin2(T/2)"]
    df["Q2eff"]=4*df["Eeff"]*df["Ep_eff"]*df["sin2(T/2)"]
    df["qv2"]=df["nu"]**2+df["Q2eff"]
    df["qv"]=np.sqrt(df["qv2"])

    df["W2"]=mass_nucleon**2+2*mass_nucleon*df["nu"]-df["Q2eff"]
    df["epsilon"]=1/(1+2*(1+(df["nu"]**2)/df["Q2eff"])*df["tan2(T/2)"]) 

    df["gamma"]=alpha_fine*df["Ep_eff"]*(df["W2"]-mass_nucleon**2)/((4*((np.pi)**2)*df["Q2eff"]*mass_nucleon*df["E0"])*(1-df["epsilon"]))
    df["Sig_R"]=df["normCross"]/df["gamma"]
    df["D_sig_R"]=df["error"]/df["gamma"]
    df["Sig_mott"]=4*(alpha_fine**2)*(df["Ep"]**2)*df["cos2(T/2)"]/(df["Q2"]**2)
    df["Sig_mott_eff"]=df["Sig_mott"]*df["E0"]/df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"]=(df["qv2"]**2)/(4*(alpha_fine**2)*(df["Ep_eff"]**2)*(df["cos2(T/2)"]+2*(df["qv2"]/df["Q2eff"])*df["sin2(T/2)"]))
    df["Hcc"]=df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"]=df["H"]*df["normCross"]
    df["Hstar_error(nb)"]=df["H"]*df["normCrossError"]
    df["Hstar_Sig(GeV)"]=df["Hstar_Sig(nb)"]/((0.1973269**2)*10000000)
    df["Hstar_error(GeV)"]=df["Hstar_error(nb)"]/((0.1973269**2)*10000000)
    df["Hcc_Sig(nb)"]=df["Hcc"]*df["normCross"]
    df["Hcc_error(nb)"]=df["Hcc"]*df["normCrossError"]
    df["Hcc_Sig(GeV)"]=df["Hcc_Sig(nb)"]/((0.1973269**2)*10000000)
    df["Hcc_error(GeV)"]=df["Hcc_error(nb)"]/((0.1973269**2)*10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"]=pd.cut(x=df["qv"],bins=qvbins,labels=qvbin_names,right=True)
    df["qvcenter"]=pd.cut(x=df["qv"],bins=qvbins,labels=qvcenters,right=True)
    df['qvcenter']=pd.to_numeric(df['qvcenter'])
    df["Q2bin"]=pd.cut(x=df["Q2eff"],bins=Q2bins,labels=Q2bin_names,right=True)
    df["Q2center"]=pd.cut(x=df["Q2eff"],bins=Q2bins,labels=Q2centers,right=True)
    # df["Q2bin"]=pd.cut(x=df["Q2"],bins=Q2bins,labels=Q2bin_names,right=True)
    # df["Q2center"]=pd.cut(x=df["Q2"],bins=Q2bins,labels=Q2centers,right=True)
    df['Q2center']=pd.to_numeric(df['Q2center'])
    df=df.dropna()
    # now every row of data has a bin (Q2/qv) label

    return df

In [8]:
# read csv file
# df = prepare_df('Data/C12_Nov12.csv')
# df = prepare_df('GEM21_11a/GEM21_11a_Nov4/GEM21_11a_Nov4.csv')
# df = pd.read_csv('df_C12_Dec10.csv')
# df = pd.read_csv('df_C12_25Feb24.csv')
# df = pd.read_csv('df_Yam_40.csv')
df = pd.read_csv('df_C12_25May23.csv')



# df = prepare_df('GENIE_LFG/G18_10a_Nov7.csv')


df

,A,Z,nu,E0,ThetaDeg,cross,error,dataSet,normalization,normError,...,RT_q2c_w2,RL_q2d_w2,RT_q2d_w2,bc_qv_nu,bc_qv_ex,bc_qv_w2,bc_q2_nu,bc_q2_ex,bc_q2_w2,sigtot
0,12,6,0.00710,0.1196,90.0,86760.0,10452.907550,1,0.9701,0.002,...,5.920377e-07,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000
1,12,6,0.01210,0.1196,90.0,26350.0,5983.562150,1,0.9701,0.002,...,6.313276e-04,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000
2,12,6,0.01710,0.1196,90.0,25810.0,4782.755457,1,0.9701,0.002,...,8.298518e-03,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000
3,12,6,0.02210,0.1196,90.0,40060.0,5031.956455,1,0.9701,0.002,...,1.017785e-02,0.006920,0.003712,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000
4,12,6,0.00210,0.1196,145.0,17100.0,978.701180,1,0.9701,0.002,...,6.229707e-09,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10719,12,6,0.29939,0.7795,50.4,2183.9,98.781211,26,1.0000,0.020,...,1.175224e-02,0.003796,0.011713,1.076481,1.029174,1.027536,1.010683,0.998638,1.012782,626.11332
10720,12,6,0.30742,0.7795,50.4,2211.5,99.026526,26,1.0000,0.020,...,1.187628e-02,0.003668,0.011858,1.112502,1.058023,1.029440,1.011040,1.000842,1.003970,729.19572
10721,12,6,0.31582,0.7795,50.4,2121.0,98.231443,26,1.0000,0.020,...,1.220422e-02,0.003554,0.012206,1.149721,1.089518,1.033198,0.999382,0.990724,0.999707,846.07368
10722,12,6,0.32345,0.7795,50.4,2148.6,98.471076,26,1.0000,0.020,...,1.264022e-02,0.003483,0.012690,1.181591,1.123927,1.038147,0.987799,0.976249,0.994606,962.17056


In [9]:
# response_columns = ['qv','q2','ex','nu','RTTOT','RLTOT','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
# response_columns = ['i','RTTOT','RLTOT']
response_columns = ['e','nu','ep','theta','ex','w2','q2','sigtot','sigqe','sigie','sigmec','nuccstot','signonuc']


# df[['qvcenter','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
df[['E0','ThetaDeg','nu']].to_csv('input.txt',index=False,header=False,sep=' ')

with open('output.txt', 'w') as output_file:
    subprocess.run(['./qemodplot_norm', 'input.txt'], stdout=output_file) 
# subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
# df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) # qvc: qv at bin center
# df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) # qvd: qv of data
print(len(df),len(df_response))
df['sigtot']=df_response['sigtot']*12*1e3




10724 10724


In [10]:
# df.to_csv('df_C12_2025Feb19.csv',index=False)
# df.to_csv('df_C12_25Feb24.csv',index=False)
# df.to_csv('df_Yam_40.csv',index=False)
df.to_csv('df_C12_25May23.csv',index=False)




In [17]:
# fake data using CB fit: prepare E0, theta, nu
df = pd.read_csv('df_C12_25Mar14.csv')
inputs = []
for E0 in np.sort(df['E0'].unique()):
    df_theta = df.loc[df['E0']==E0]
    for ThetaDeg in np.sort(df_theta['ThetaDeg'].unique()):
        nus = np.arange(0,E0,0.002)
        for nu in nus:
            inputs.append([E0,ThetaDeg,nu])
inputs=np.array(inputs)
inputs[:,0]
df_inputs = pd.DataFrame({'E0':inputs[:,0],'ThetaDeg':inputs[:,1],'nu':inputs[:,2]})
df_inputs.to_csv('input.txt',index=False,header=False,sep=' ')
df_inputs

,E0,ThetaDeg,nu
0,0.054,180.0,0.000
1,0.054,180.0,0.002
2,0.054,180.0,0.004
3,0.054,180.0,0.006
4,0.054,180.0,0.008
...,...,...,...
158909,5.766,32.0,5.756
158910,5.766,32.0,5.758
158911,5.766,32.0,5.760
158912,5.766,32.0,5.762


In [18]:
# run sigtot for fake data
response_columns = ['e','nu','ep','theta','ex','w2','q2','sigtot','sigqe','sigie','sigmec','nuccstot','signonuc']

with open('output.txt', 'w') as output_file:
    subprocess.run(['./qemodplot_norm', 'input.txt'], stdout=output_file) 
# subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
# df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) # qvc: qv at bin center
# df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) # qvd: qv of data


In [19]:
df_response['E0']=df_response['e']
df_response['ThetaDeg']=df_response['theta']
df_response['A']=12
df_response['Z']=6
df_response['cross']=df_response['sigtot']*12*1e3
df_response[['A','Z','E0','ThetaDeg','nu','cross']].to_csv('C12-cross-section-CBfit.csv',index=False)


In [8]:
# calculate bc_qv_nu

# response_columns = ['qv','q2','ex','nu','RTTOT','RLTOT','RTQE','RLQE','RTIE','RLIE','RTE','RLE','RTNS','RLNS']
# response_columns = ['i','RTTOT','RLTOT']
response_columns = ['ep','theta','ex','w2','q2','sigtot','sigqe','sigie','sigmec','nuccstot','signonuc']


# df[['qvcenter','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
df[['E0','ThetaDeg']].to_csv('input.txt',index=True,header=False,sep=' ')

with open('output.txt', 'w') as output_file:
    subprocess.run(['./qemodplot_norm', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) # qvc: qv at bin center
df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) # qvd: qv of data







df[['qv','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvd_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])

print('RL RT qv_nu done.')




# calculate bc_qv_ex
df[['qvcenter','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvc_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvd_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_ex done.')




# calculate bc_qv_w2
df[['qvcenter','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvc_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_qvd_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

print('RL RT qv_w2 done.')



# calculate bc_q2_nu
df[['Q2center','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2c_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2d_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])

print('RL RT q2_nu done.')



# calculate bc_q2_ex
df[['Q2center','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2c_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','Ex']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2d_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

print('RL RT q2_ex done.')




# calculate bc_q2_w2
df[['Q2center','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2c_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
df['RL_q2d_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

print('RL RT q2_w2 done.')




RL RT qv_nu done.
RL RT qv_ex done.
RL RT qv_w2 done.
RL RT q2_nu done.
RL RT q2_ex done.
RL RT q2_w2 done.


In [9]:
# determine bin-centering corrections
df["bc_qv_nu"]=1.0
df["bc_qv_ex"]=1.0
df["bc_qv_w2"]=1.0
df["bc_q2_nu"]=1.0
df["bc_q2_ex"]=1.0
df["bc_q2_w2"]=1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"]==qvcenter]
    for index, row in picked.iterrows():
        if row['qvcenter']!= float('NaN') and row['Ex']>=0.025:
            try:
               df.loc[index, 'bc_qv_nu']= (row['epsilon']*row['RL_qvc_nu']+0.5*((qvcenter**2)/(qvcenter**2-row['nu']**2))*row['RT_qvc_nu']
                    )/(row['epsilon']*row['RL_qvd_nu']+0.5*(row['qv2']/row['Q2eff'])*row['RT_qvd_nu'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index)
            
            try:
                df.loc[index, 'bc_qv_ex']= (row['epsilon']*row['RL_qvc_ex']+0.5*((qvcenter**2)/(qvcenter**2-row['nu']**2))*row['RT_qvc_ex']
                    )/(row['epsilon']*row['RL_qvd_ex']+0.5*(row['qv2']/row['Q2eff'])*row['RT_qvd_ex'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index)
            
            try:
               df.loc[index, 'bc_qv_w2']= (row['epsilon']*row['RL_qvc_w2']+0.5*((qvcenter**2)/(2*mass_nucleon*np.sqrt(qvcenter**2+row['W2'])-row['W2']-mass_nucleon**2)
                                                    )*row['RT_qvc_w2'])/(row['epsilon']*row['RL_qvd_w2']+0.5*(row['qv2']/row['Q2eff'])*row['RT_qvd_w2'])
            except ZeroDivisionError:
                print('ZeroDivisionError, index:',index)

for Q2center in Q2centers:
    picked = df.loc[df["Q2center"]==Q2center]
    for index, row in picked.iterrows():
        if row['Q2center']!= float('NaN') and row['Ex']>=0.025:
            df.loc[index, 'bc_q2_nu']= (row['epsilon']*row['RL_q2c_nu']+0.5*((Q2center+row["nu"]**2)/(Q2center))*row['RT_q2c_nu']
                    )/(row['epsilon']*row['RL_q2d_nu']+0.5*(row['qv2']/row['Q2eff'])*row['RT_q2d_nu'])
            
            df.loc[index, 'bc_q2_ex']= (row['epsilon']*row['RL_q2c_ex']+0.5*((Q2center+row["nu"]**2)/(Q2center))*row['RT_q2c_ex']
                    )/(row['epsilon']*row['RL_q2d_ex']+0.5*(row['qv2']/row['Q2eff'])*row['RT_q2d_ex'])

            nu = (row['W2']+Q2center-mass_nucleon**2)/(2*mass_nucleon)
            qv2center = Q2center+nu**2
            df.loc[index, 'bc_q2_w2']= (row['epsilon']*row['RL_q2c_w2']+0.5*(qv2center/(Q2center))*row['RT_q2c_w2']
                    )/(row['epsilon']*row['RL_q2d_w2']+0.5*(row['qv2']/row['Q2eff'])*row['RT_q2d_w2'])

ZeroDivisionError, index: 17887
ZeroDivisionError, index: 17887
ZeroDivisionError, index: 2408
ZeroDivisionError, index: 2408
ZeroDivisionError, index: 2570
ZeroDivisionError, index: 2570


In [10]:
# save dataframe to csv
df.to_csv('GEM21_11a/GEM21_11a_Nov4/df_GEM21_11a_Nov4.csv',index=False)
# df.to_csv('GENIE_LFG/df_G18_10a_Nov7.csv',index=False)


